# Генерация текста с помощью LSTM-сетей

Сеть способна выучить распределение символов в последовательностях


Датасет формируем проходясь окном по текстовому корпусу, задача сети - предсказывать следующий символ на основании нескольких предыдущих.
Данный подход можно улучшить, используя только отдельные предложения с паддингами.

In [1]:
import torch
import torch.nn as nn

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 0. Получение данных для обучения

Для обучения используется датасет российских новостей, который был сохранён в файл `text_corpus.parquet` со следующими параметрами:

In [12]:
# data.to_parquet("text_corpus.parquet", engine="pyarrow", compression="gzip", index=False)

In [3]:
data = pd.read_parquet("text_corpus.parquet", engine="pyarrow", )

In [4]:
len(data)

50000

In [5]:
# после обучения токенизатора можно уменьшить тренировочную выборку
# но нужно не забыть обновить переменную corpus
data = data.sample(10000)

### 1. Вспомогательные функции:
+ Визуализация процесса обучения
    + Сможем посмотреть, как меняется качество с течением времени.
+ Коллбек ModelCheckpoint
    + Процесс обучения LSTM сетей достаточно длительный. Будет обидно, если из-за непредвиденного сбоя потеряется прогресс за многие часы обучения.
+ Колбек динамической подстройки размера батча и learning rate
    + Подстраивать LR это уже стандартная практика, а я хочу ещё и размер батча менять: предположу, что большой батч позволит дать некое "обобщённое" представление о распределении токенов, а маленький батч улучшит "грамотность".



In [6]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

In [9]:
# весь текст одной "портянкой", чтобы заранее оценить, какие символы могут нам попадаться
# raw_text = " ".join(data.text)
# chars = sorted(list(set(raw_text)))
# chars

### 3. Предобработка и создание датасета

Для тренировки LSTM модели понадобится немного поработать с форматами

In [7]:
import re

In [8]:
corpus = " \n".join(data.text.to_list()).lower()
# Хочу отделить всю пунктуацию от слов пробелом
corpus = " ".join(re.findall(r"[\w']+|[.,!?;\n]", corpus))

In [11]:
total_words = 800
max_sequence_length = 80
n_samples = 8000000 # сколько тренировочных последовательностей потом сгенерируем из корпуса

#### 3.1 Токенизация BPE 

BPE токенизация посредством yttm эффективна, но потребуется поработать с файлом

In [9]:
import youtokentome as yttm

In [10]:
bpe_model_path = "bpe.yttm"

##### 3.2 Создаём токенизатор BPE и обучаем его

In [12]:
def create_bpe_tokenizer_from_scratch(corpus, train_data_path="yttm_train_data.txt"):
    with open(train_data_path, "w") as _file:
        _file.writelines(corpus)
    # Training model
    # (data, model, vocab_size, coverage, n_threads=-1, pad_id=0, unk_id=1, bos_id=2, eos_id=3)
    return yttm.BPE.train(data=train_data_path, vocab_size=total_words, model=bpe_model_path)

In [21]:
# Creating model
bpe = create_bpe_tokenizer_from_scratch(corpus)

In [13]:
# Loading model
bpe = yttm.BPE(model=bpe_model_path)

In [14]:
print(' '.join(bpe.vocab())[:300])

<PAD> <UNK> <BOS> <EOS> ▁ о е и а н т с р в л к п д м у я ы г з б , ь ч й . х ж ' ц ю ш ф щ э ъ ? ё ! ; _ ▁п ▁с ▁в ▁, ст ни ра ро но ре на ▁о ко то ▁. ▁и ▁по го не де те ли ва ▁м за ны ▁на ль ка ри та ле ла ▁д во ве ▁б ти ци ▁со ви ▁ч ки ло ▁у ▁за ▁' да ть ен ми ▁а ▁не ▁ко сс ▁пре ет ру ся ди ▁про н


In [102]:
# encode(self, 
#     sentences, 
#     output_type=yttm.OutputType.ID, 
#     bos=False, 
#     eos=False, 
#     reverse=False, 
#     dropout_prob=0)

In [18]:
encoded_corpus = np.array(bpe.encode(corpus))

# sequences = sequence[:-(len(sequence)%max_sequence_length)].reshape((len(sequence)//max_sequence_length, max_sequence_length))

## 4. Модель

В качестве модели будет применяться LSTM сеть с двумя слоями LSTM

TODO
+ Gradient clipping
+ More layers?

In [22]:
class LSTMModel(nn.Model)
    def __init__(
            self, 
            input_size=max_input_length,
            num_classes=total_words,
            hidden_dim=64,
            num_layers=2,
            batch_size=128,
                ):
        super(LSTMModel, self).__init__()
        self.input_size = input_size
        self.num_classes = num_classes
        # Dropout
        self.dropout = nn.Dropout(0.25)
        
        # Embedding layer
        self.embedding = nn.Embedding(self.input_size, self.hidden_dim, padding_idx=0)
        # Bi-LSTM
        # Forward and backward
        self.lstm_cell_forward = nn.LSTMCell(self.hidden_dim, self.hidden_dim, batch_first=True)
        self.lstm_cell_backward = nn.LSTMCell(self.hidden_dim, self.hidden_dim, batch_first=True)
        # LSTM layer
        self.lstm_cell = nn.LSTMCell(self.hidden_dim * 2, self.hidden_dim * 2, batch_first=True)
        
#         self.lstm = nn.LSTM(
#             max_input_length,  # input_size – The number of expected features in the input x
#             hidden_dim, # hidden_size – The number of features in the hidden state h
#             num_layers, # num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1
#             # bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
#             batch_first=True# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False
#             # dropout – If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0
#             bidirectional=True# bidirectional – If True, becomes a bidirectional LSTM. Default: False
#             # proj_size – If > 0, will use LSTM with projections of corresponding size. Default: 0
#         )
        
        self.linear = nn.Linear(self.hidden_dim * 2, self.num_classes)
        
        
    def forward(self, X):
        # Bi-LSTM
        # hs = [batch_size x hidden_size]
        # cs = [batch_size x hidden_size]
        hs_forward = torch.zeros(X.size(0), self.hidden_dim)
        cs_forward = torch.zeros(X.size(0), self.hidden_dim)
        hs_backward = torch.zeros(X.size(0), self.hidden_dim)
        cs_backward = torch.zeros(X.size(0), self.hidden_dim)

        # LSTM
        # hs = [batch_size x (hidden_size * 2)]
        # cs = [batch_size x (hidden_size * 2)]
        hs_lstm = torch.zeros(X.size(0), self.hidden_dim * 2)
        cs_lstm = torch.zeros(X.size(0), self.hidden_dim * 2)

        # Weights initialization
        torch.nn.init.kaiming_normal_(hs_forward)
        torch.nn.init.kaiming_normal_(cs_forward)
        torch.nn.init.kaiming_normal_(hs_backward)
        torch.nn.init.kaiming_normal_(cs_backward)
        torch.nn.init.kaiming_normal_(hs_lstm)
        torch.nn.init.kaiming_normal_(cs_lstm)

        # From idx to embedding
        out = self.embedding(X)

        # Prepare the shape for LSTM Cells
#         out = out.view(self.sequence_len, X.size(0), -1)

        forward = []
        backward = []

        # Unfolding Bi-LSTM
        # Forward
        for i in range(self.input_size):
            hs_forward, cs_forward = self.lstm_cell_forward(out[i], (hs_forward, cs_forward))
            hs_forward = self.dropout(hs_forward)
            cs_forward = self.dropout(cs_forward)
            forward.append(hs_forward)

         # Backward
        for i in reversed(range(self.sequence_len)):
            hs_backward, cs_backward = self.lstm_cell_backward(out[i], (hs_backward, cs_backward))
            hs_backward = self.dropout(hs_backward)
            cs_backward = self.dropout(cs_backward)
            backward.append(hs_backward)

         # LSTM
        for fwd, bwd in zip(forward, backward):
            input_tensor = torch.cat((fwd, bwd), 1)
            hs_lstm, cs_lstm = self.lstm_cell(input_tensor, (hs_lstm, cs_lstm))

         # Last hidden state is passed through a linear layer
        out = self.linear(hs_lstm)

        return out

In [23]:
def train(self, args):
  
      # Model initialization
      model = TextGenerator(args, self.vocab_size)

      # Optimizer initialization
      optimizer = optim.RMSprop(model.parameters(), lr=self.learning_rate)

      # Defining number of batches
      num_batches = int(len(self.sequences) / self.batch_size)

      # Set model in training mode
      model.train()

      # Training pahse
      for epoch in range(self.num_epochs):

            # Mini batches
            for i in range(num_batches):

                  # Batch definition
                try:
                    x_batch = self.sequences[i * self.batch_size : (i + 1) * self.batch_size]
                    y_batch = self.targets[i * self.batch_size : (i + 1) * self.batch_size]
                except:
                    x_batch = self.sequences[i * self.batch_size :]
                    y_batch = self.targets[i * self.batch_size :]

                # Convert numpy array into torch tensors
                x = torch.from_numpy(x_batch).type(torch.LongTensor)
                y = torch.from_numpy(y_batch).type(torch.LongTensor)

                # Feed the model
                y_pred = model(x)

                # Loss calculation
                loss = F.cross_entropy(y_pred, y.squeeze())

                # Clean gradients
                optimizer.zero_grad()

                # Calculate gradientes
                loss.backward()

                # Updated parameters
                optimizer.step()

                print("Epoch: %d ,  loss: %.5f " % (epoch, loss.item()))

In [ ]:
# Save weights
torch.save(model.state_dict(), 'weights/textGenerator_model.pt')

## 5. Инференс полученной модели

In [102]:
def generator(model, sequences, idx_to_char, n_chars):
  
    # Set the model in evalulation mode
    model.eval()

    # Define the softmax function
    softmax = nn.Softmax(dim=1)

    # Randomly is selected the index from the set of sequences
    start = np.random.randint(0, len(sequences)-1)

    # The pattern is defined given the random idx
    pattern = sequences[start]

    # By making use of the dictionaries, it is printed the pattern
    print("\nPattern: \n")
    print(''.join([idx_to_char[value] for value in pattern]), "\"")

    # In full_prediction we will save the complete prediction
    full_prediction = pattern.copy()

    # The prediction starts, it is going to be predicted a given
    # number of characters
    for i in range(n_chars):

        # The numpy patterns is transformed into a tesor-type and reshaped
        pattern = torch.from_numpy(pattern).type(torch.LongTensor)
        pattern = pattern.view(1,-1)

        # Make a prediction given the pattern
        prediction = model(pattern)
        # It is applied the softmax function to the predicted tensor
        prediction = softmax(prediction)

        # The prediction tensor is transformed into a numpy array
        prediction = prediction.squeeze().detach().numpy()
        # It is taken the idx with the highest probability
        arg_max = np.argmax(prediction)

        # The current pattern tensor is transformed into numpy array
        pattern = pattern.squeeze().detach().numpy()
        # The window is sliced 1 character to the right
        pattern = pattern[1:]
        # The new pattern is composed by the "old" pattern + the predicted character
        pattern = np.append(pattern, arg_max)

        # The full prediction is saved
        full_prediction = np.append(full_prediction, arg_max)

    print("Prediction: \n")
    print(''.join([idx_to_char[value] for value in full_prediction]), "\"")

приветствие затянулось на несколько вродители,
им зае призиденторта согорами в сосесуми рроведе с сосрийской ке просева об пнобо с общела он вобороднтое влодедать в инсотия чесьдае в сосрий сазорунотти в соссий сакали проведа пода саков


In [120]:
output_character

['<PAD>']

In [72]:
bpe.decode([list(X[522].reshape((max_sequence_length-1)))])

["не планирует передислоцировать наблюдательные пункты в идлибской зоне деэскалации , при этом турция продолжит отправлять военных и бронетехнику в этот район 'в целях защиты мирного населения' . ка"]

In [28]:
# Для bpe
composition = "не планирует передислоцировать наблюдательные пункты в идлибской зоне деэскалации , при этом турция продолжит отправлять военных и бронетехнику в этот район 'в целях защиты мирного населения' . ка"
next_words = 200
  
for _ in range(next_words):
    token_list = bpe.encode(composition)
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre', truncating="pre")
    token_list = token_list.reshape((1,max_sequence_length-1,1))
    predicted = np.argmax(model2.predict(token_list), axis=-1)
    output_character = bpe.decode([predicted])[0]
    composition += output_character
print(composition)

не планирует передислоцировать наблюдательные пункты в идлибской зоне деэскалации , при этом турция продолжит отправлять военных и бронетехнику в этот район 'в целях защиты мирного населения' . ка,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [57]:
bpe.encode(["приветствие затянулось на несколько "])

[[126, 341, 322, 6, 90, 10, 20, 156, 603, 71, 97, 109, 448]]

In [44]:
import random

In [45]:
def return_ordered_indices(ar):
    d = {i:v for i,v in enumerate(ar)}
    return sorted(d, key=d.get, reverse=True)

In [46]:
# ensures always sums to 1
def normalize_softmax(ar):
    s = sum(ar)
    if (s!=1):
        ar[0] += 1-s
    return ar
        

In [47]:
composition = "как было сказано "
next_words = 200
T = 2 # токены из top-T будут случайно выбираться
temperature = 1 # параметр сглаживания распределения выбранных токенов
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([composition])[0]
    token_list = pad_sequences([token_list], maxlen=window_length-1, padding='pre')
    token_list = token_list.reshape((1,window_length-1,1))
    
    output = model2.predict(token_list)
    topmost_indicies = return_ordered_indices(output[0, :])[:T]
    probs = tf.nn.softmax(output[0, topmost_indicies] /  temperature).numpy()
    probs = normalize_softmax(probs)
    predicted = np.random.choice(topmost_indicies, p=probs)
#     predicted = topmost_indicies[0]
    output_character = tokenizer.sequences_to_texts([[predicted]])[0]
    composition += output_character
print(composition)

как было сказано осенее с онкет пантоти и ресетиеее ес о осессон в паттеле онти портовония сакомо презедлитеви презисселе нерари соргорам сосриий,
подономо по накраветения подоваи полодать по накогния сообщал возении,
